# [에어크루] 항공사 고객 만족도 예측 경진대회 
# Stage 2. 데이터 전처리

항공사 고객 만족도 예측 경진대회에 참여하신 데이커 여러분, 환영합니다. 🙌  
이번 시간에는 패키지를 설치하고 본격적으로 분석 및 모델링을 진행하기 전에  필요한 데이터 전처리를 진행해보려 합니다.

---

## 1. 패키지 불러오기

### 1.1 필요한 패키지 확인

데이터 전처리에 앞서, 필요한 패키지를 불러오겠습니다. 

기본적으로 사용하는 pandas, matplotlib, numpy, seaborn을 ‘import’하면 바로 사용할 수 있어요!

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns

---

## 2. 데이터 전처리

### 2.0 데이터 열 선택

train, test 데이터에서 필요 없는 열은 삭제하고 저희가 예측할 target 열은 train_y로 따로 빼두겠습니다. drop 메소드를 이용해 삭제하고 싶은 열을 선택하면 삭제 완료!

In [3]:
train_x = train.drop(['id','target'],axis=1) 
train_y = train.target 
test_x = test.drop(['id'],axis = 1)

### 2.1 데이터 인코딩

**데이터 라벨 인코딩** 왜 필요할까요? 

object type의 데이터를 모델에 넣으면 모델이 데이터를 인식하지 못하기 때문에 에러가 발생합니다! 때문에 수치형 데이터가 아니면, 라벨 인코딩은 필수라는 점!

라벨 인코딩이란 n개의 범주형 데이터를 0부터 (n-1)까지의 연속적인 수치 데이터로 표현하는 과정 입니다.

그렇다면 라벨 인코딩이 필요한 열을 알아볼까요?

1. Gender → [Female, Male]
2. Customer Type → ['disloyal Customer', 'Loyal Customer']
3. Type of Travel → ['Business travel', 'Personal Travel’]
4. Class → ['Eco', 'Business', 'Eco Plus’]

해당 열을 따로 변수로 빼두고 라벨 인코딩을 진행해보겠습니다.

In [5]:
from sklearn.preprocessing import LabelEncoder

# qualitative to quantitative
qual_col = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #train_x와 test_x의 Label이 다르게 Encoding될 우려가 있어 train_x로 fit을 고정
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


### 2.2. 데이터 타입 변환

데이터 타입 변환하는 방법도 알려드릴게요!

train_x 데이터의 info를 확인해보면 Arrival Delay in Minute 열의 데이터가 float(실수형)인 것을 확인할 수 있습니다. 나머지 열은 전부 int (정수형)이니 Arrival Delay in Minute 도 int로 바꿔주겠습니다.

In [8]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             3000 non-null   int32  
 1   Customer Type                      3000 non-null   int32  
 2   Age                                3000 non-null   int64  
 3   Type of Travel                     3000 non-null   int32  
 4   Class                              3000 non-null   int32  
 5   Flight Distance                    3000 non-null   int64  
 6   Seat comfort                       3000 non-null   int64  
 7   Departure/Arrival time convenient  3000 non-null   int64  
 8   Food and drink                     3000 non-null   int64  
 9   Gate location                      3000 non-null   int64  
 10  Inflight wifi service              3000 non-null   int64  
 11  Inflight entertainment             3000 non-null   int64

데이터 타입을 바꾸고 싶은 열과 바꾸고 싶은 데이터 타입을 아래와 같이  astype을 이용해 입력하면 됩니다!

In [10]:
train_x = train_x.astype({'Arrival Delay in Minutes':'int'})

### 2.3. 정규화

마지막으로 정규화입니다.

정규화는 모든 값을 0~1 사이의 값으로 바꾸는 과정입니다. 

데이터를 확인해보면 Flight Distance는 1599, 2810 … 으로 1000 자리가 넘어가면서 다른 데이터들과 차이가 큰 것을 확인할 수 있어요! 이를 무시하고 모델을 진행할 경우 성능이 낮아질 수 있습니다. 

MinMaxScaler를 활용해 정규화를 진행해보겠습니다.

In [11]:
from sklearn.preprocessing import StandardScaler

num_features = ['Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes']

scaler = StandardScaler()

#Data Leakage가 일어나지 않도록 train_x를 통해 스케일러를 학습 시키고 (fit)
scaler.fit(train_x[num_features])

#만들어진 스케일러로 train_x를 정규화해줍니다.(transform)
train_x[num_features] = scaler.transform(train_x[num_features])

train_x.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,0,1,22,0,1,-0.373641,3,0,3,3,...,4,4,5,4,4,4,5,4,-0.346846,-0.352289
1,0,0,37,0,0,0.804446,2,4,4,4,...,3,5,5,4,2,1,5,2,0.052482,0.045978
2,1,0,46,0,0,0.621556,1,1,1,1,...,5,4,4,4,4,5,4,3,-0.346846,-0.352289
3,0,1,24,0,1,0.355003,3,3,3,3,...,3,3,2,4,5,3,4,3,-0.124997,-0.308037
4,0,0,58,0,0,-1.827036,3,3,3,3,...,5,4,4,4,4,4,4,5,-0.346846,-0.352289


---

stage 2, 무사히 완료 =33

다음 stage에서 만나요!